In [1]:
import matplotlib.pyplot as plt
import scienceplots
import time
import edist.ted as ted
import joblib

# try to load data
import os

import hyperelastic_laws as hyperelastic
import torch
import sympy 
from sympy import Pow, MatrixSymbol, Trace, log, MatMul
import numpy as np
import symengine as se
from sympy.parsing.sympy_parser import parse_expr

import tree

import cma

import recursive_tree_grammar_auto_encoder as rtg_ae

In [2]:
plt.rcParams.update(plt.rcParamsDefault)
plt.rc('font', family='serif')
plt.rcParams.update({
                      "text.usetex": True,
                      "font.family": "serif",
                      'text.latex.preamble': r'\usepackage{amsmath}',
                      'font.size': 20,
                      'lines.linewidth': 3,
                      'axes.labelsize': 22,  
                      'axes.titlesize': 24,
                      'xtick.labelsize': 20,
                      'ytick.labelsize': 20,
                      'legend.fontsize': 20,
                      'axes.linewidth': 2})

In [3]:
dim = 128
dim_vae  = 8
num_params = 0
model = rtg_ae.TreeGrammarAutoEncoder(hyperelastic.grammar, dim = dim, dim_vae = dim_vae)
for key in model.state_dict():
    num_params += model.state_dict()[key].numel()
print('The model contains %d parameters' % (num_params))

The model contains 402080 parameters


In [4]:
def to_algebraic_string(nodes, adj, i = 0):
    if list(',') in list(map(str.split, nodes[i])):
        return list(map(str.split, str(nodes[i])))[0][0] + '.'+ list(map(str.split, str(nodes[i])))[2][0]
    if nodes[i] == '+' or nodes[i] == '*' or nodes[i] == '-'  or nodes[i] == '/':
        return to_algebraic_string(nodes, adj, adj[i][0]) + ' ' + nodes[i] + ' ' + to_algebraic_string(nodes, adj, adj[i][1])
    if nodes[i] == 'log' or nodes[i] == 'exp':
        return nodes[i] + '(' + to_algebraic_string(nodes, adj, adj[i][0]) + ')'
    if nodes[i] == 'pow':
        return '(' + to_algebraic_string(nodes, adj, adj[i][0]) + ')' + '**' + to_algebraic_string(nodes, adj, adj[i][1])
    if nodes[i] == '(J-1)':
        return '(' + nodes[i] + ')' 
    if nodes[i] == '(I1-3)':
        return '(' + 'I1-3' + ')' 
    if nodes[i] == '(I2-3)':
        return '(' + 'I2-3' + ')'
    else:
        return nodes[i]

def evaluate_se(nodes, adj, I1_tilde, I2_tilde, J_tilde, F11_tilde, F22_tilde, F33_tilde, train=False):
    J, I1, I2 = se.symbols("J I1 I2")
    sympy_expression = se.sympify(to_algebraic_string(nodes,adj))
    y_all = np.zeros_like(I1_tilde)
    for i in range(0, I1_tilde.shape[0]):
        y_all[i] = np.array(sympy_expression.subs({J:se.Pow(se.sqrt(J_tilde[i]),2), I1:se.Pow(se.sqrt(I1_tilde[i]),2), I2:se.Pow(se.sqrt(I2_tilde[i]),2)}).evalf(16)).astype(np.float128)
    return y_all

def objective_function(nodes, adj, I1_tilde, I2_tilde, J_tilde, F11_tilde, F22_tilde, F33_tilde, y):
    y_pred = evaluate_se(nodes, adj, I1_tilde, I2_tilde, J_tilde, F11_tilde, F22_tilde, F33_tilde, train=True)
    return np.sqrt(np.mean((y - y_pred) ** 2 ))

In [5]:

# model_name = 'Neo-Hookean'
# model_name = 'Isihara'
# model_name = 'Haines-Wilson'
# model_name = 'Gent-Thomas'
model_name = 'Ogden'

invalid_const = 1000.

def Neo_Hookean(I1, I2, J):
    return 0.5*(I1 - 3) + 1.5*(J - 1)**2

def Isihara(I1, I2, J):
    return 0.5*(I1 - 3) + (I2 - 3) + (I1 - 3)**2 + 1.5*(J-1)**2

def Haines_Wilson(I1, I2, J):
    return 0.5*(I1 - 3) + (I2 - 3) + 0.7*(I1 - 3)*(I2 - 3) + 0.2*(I1 - 3)**3 + 1.5*(J-1)**2

def Gent_Thomas(I1, I2, J):
    return 0.5*(I1 - 3) + np.log(I2/3) + 1.5*(J-1)**2

def Ogden(I1, I2, J):
        kappa_ogden = 1.5
        mu_ogden = 0.65
        alpha_ogden = 0.65
        I1_tilde = I1 + 1e-13
        I1t_0 = np.array([3]) + 1e-13
        J_0 = np.array([1]) + 1e-13
        W_offset = kappa_ogden*(J_0-1)**2 + 1/alpha_ogden * 2. * (0.5**alpha_ogden*(I1t_0  +  np.sqrt(  (I1t_0-1/(J_0**(2./3.)))**2 - 4*J_0**(2./3.)) - 1/(J_0**(2./3.)) )**alpha_ogden+( 0.5*I1t_0 - 0.5*np.sqrt(  (I1t_0-1/(J_0**(2./3.)))**2 - 4*J_0**(2./3.))  - 0.5/(J_0**(2./3.)) )**alpha_ogden + J_0**(-alpha_ogden*2./3.) ) * mu_ogden
        W_truth = kappa_ogden*(J-1)**2 + 1/alpha_ogden * 2. * (0.5**alpha_ogden*(I1_tilde  +  np.sqrt(  (I1_tilde-1/(J**(2./3.)))**2 - 4*J**(2./3.)) - 1/(J**(2./3.)) )**alpha_ogden+( 0.5*I1_tilde - 0.5*np.sqrt(  (I1_tilde-1/(J**(2./3.)))**2 - 4*J**(2./3.))  - 0.5/(J**(2./3.)) )**alpha_ogden + J**(-alpha_ogden*2./3.) ) * mu_ogden - W_offset
        return W_truth

def decoding_functions(h):
    nodes, adj, _ = model.decode(torch.tensor(h, dtype=torch.float), max_size = 2*11)
    return nodes, adj

save_file_Neo_Hookean30  = '/home/gkissas/GrammarVAEs/rtgae-master 2/' + 'NeoHookean30_noise1e0'
save_file_Isihara30      = '/home/gkissas/GrammarVAEs/rtgae-master 2/' + 'Isihara30_noise1e0'
save_file_Haine_Wilson30 = '/home/gkissas/GrammarVAEs/rtgae-master 2/' + 'HainesWilson30_noise1e0'
save_file_Gent_Thomas30  = '/home/gkissas/GrammarVAEs/rtgae-master 2/' + 'GentThomas30_noise1e0'

save_file_Ogden30  = '/home/gkissas/GrammarVAEs/rtgae-master 2/' + 'Ogden30_noise1e0'

if model_name == "Neo-Hookean":
    d30 = np.load(save_file_Neo_Hookean30 +'.npz')
    f = Neo_Hookean

elif model_name == "Isihara":
    d30 = np.load(save_file_Isihara30 +'.npz')
    f = Isihara

elif model_name == "Haines-Wilson":
    d30 = np.load(save_file_Haine_Wilson30 +'.npz')
    f = Haines_Wilson

elif model_name == "Gent-Thomas":
    d30 = np.load(save_file_Gent_Thomas30 +'.npz')
    f = Gent_Thomas

elif model_name == "Ogden":
    d30 = np.load(save_file_Ogden30 +'.npz')
    f = Ogden

J   = d30['J'][:,0]
I1  = d30['I1'][:,0]
I2  = d30['I2'][:,0]


F11  = d30['F'][:,0]
F22  = d30['F'][:,3]
F33  = np.ones_like(F11)

I1 = J**(-2/3)*I1
I2 = J**(-4/3)*I2
y = f(I1, I2, J)

In [6]:
model.load_state_dict(torch.load('results/saved_model_hyperelastic.torch'))

<All keys matched successfully>

In [7]:
Jarray  = J
I1array = I1
I2array = I2

In [8]:
def get_I1(J, C):
    I1 = Pow(J,-2/3)*(C[0,0] + C[1,1] + 1.)
    return I1

def get_I2(J, C):
    I2 = Pow(J,-4/3)*(C[0,0] + C[1,1] -C[0,1]*C[1,0] + C[0,0]*C[1,1])
    return I2

def get_Ogden(J, C):
    kappa_ogden = 1.5
    mu_ogden = 0.65
    alpha_ogden = 0.65
    I1 = C[0,0] + C[1,1] + 1.
    I1_tilde = J**(-2/3)*I1 + 0.0000001
    I1t_0 = 3 + 0.00000001
    J_0 = 1 + 0.0000001
    W_offset = kappa_ogden*(J_0-1)**2 + 1/alpha_ogden * 2. * (0.5**alpha_ogden*(I1t_0  +  sympy.sqrt(  (I1t_0-1/(J_0**(2./3.)))**2 - 4*J_0**(2./3.)) - 1/(J_0**(2./3.)) )**alpha_ogden+( 0.5*I1t_0 - 0.5*sympy.sqrt(  (I1t_0-1/(J_0**(2./3.)))**2 - 4*J_0**(2./3.))  - 0.5/(J_0**(2./3.)) )**alpha_ogden + J_0**(-alpha_ogden*2./3.) ) * mu_ogden
    #% Final output
    W_truth = kappa_ogden*(J-1)**2 + 1/alpha_ogden * 2. * (0.5**alpha_ogden*(I1_tilde  +  sympy.sqrt(  (I1_tilde-1/(J**(2./3.)))**2 - 4*J**(2./3.)) - 1/(J**(2./3.)) )**alpha_ogden+( 0.5*I1_tilde - 0.5*sympy.sqrt(  (I1_tilde-1/(J**(2./3.)))**2 - 4*J**(2./3.))  - 0.5/(J**(2./3.)) )**alpha_ogden + J**(-alpha_ogden*2./3.) ) * mu_ogden - W_offset
    return W_truth

def get_Ogden_s(J, C, I1):
    kappa_ogden = 1.5
    mu_ogden = 0.65
    alpha_ogden = 0.65
    I1_tilde = I1 + 0.0000001
    I1t_0 = 3 + 0.00000001
    J_0 = 1 + 0.0000001
    W_offset = kappa_ogden*(J_0-1)**2 + 1/alpha_ogden * 2. * (0.5**alpha_ogden*(I1t_0  +  sympy.sqrt(  (I1t_0-1/(J_0**(2./3.)))**2 - 4*J_0**(2./3.)) - 1/(J_0**(2./3.)) )**alpha_ogden+( 0.5*I1t_0 - 0.5*sympy.sqrt(  (I1t_0-1/(J_0**(2./3.)))**2 - 4*J_0**(2./3.))  - 0.5/(J_0**(2./3.)) )**alpha_ogden + J_0**(-alpha_ogden*2./3.) ) * mu_ogden
    #% Final output
    W_truth = kappa_ogden*(J-1)**2 + 1/alpha_ogden * 2. * (0.5**alpha_ogden*(I1_tilde  +  sympy.sqrt(  (I1_tilde-1/(J**(2./3.)))**2 - 4*J**(2./3.)) - 1/(J**(2./3.)) )**alpha_ogden+( 0.5*I1_tilde - 0.5*sympy.sqrt(  (I1_tilde-1/(J**(2./3.)))**2 - 4*J**(2./3.))  - 0.5/(J**(2./3.)) )**alpha_ogden + J**(-alpha_ogden*2./3.) ) * mu_ogden - W_offset
    return W_truth


F = MatrixSymbol("F",3,3)
J = sympy.Determinant(F.as_explicit()) 
C = MatMul(F.as_explicit().T,F.as_explicit())
I1 = get_I1(J, C)
I2 = get_I2(J, C)

Neo_Hookean = 0.5*(I1 - 3) + 1.5*Pow(J - 1,2)
Isihara = 0.5*(I1 - 3) + (I2 - 3) + Pow(I1 - 3,2) + 1.5*(Pow(J-1, 2))
Haines_Wilson = 0.5*(I1 - 3) + (I2 - 3) + 0.7*(I1 - 3)*(I2 - 3) + 0.2*(Pow(I1 - 3,3)) + 1.5*(Pow(J-1,2))
Gent_Thomas = 0.5*(I1 - 3) + log(I2/3) + 1.5*(Pow(J-1,2)) 

Ogden = get_Ogden(J,C)

J_s, C_s, I1_s = sympy.symbols('J C I1')
Ogden_invariants = get_Ogden_s(J_s, C_s, I1_s)

In [10]:
y_ogden_base = y

In [11]:
NH_baseline = '0.5*(I1 - 3) + 1.5*(J - 1)**2'
IS_baseline = '0.5*(I1 - 3) + (I2 - 3) + (I1 - 3)**2 + 1.5*(J-1)**2'
HW_baseline = '0.5*(I1 - 3) + (I2 - 3) + 0.7*(I1 - 3)*(I2 - 3) + 0.2*(I1 - 3)**3 + 1.5*(J-1)**2'
GT_baseline = '0.5*(I1 - 3) + log(I2/3) + 1.5*(J-1)**2'

NH_prediction_noise1e0 = "0.5*(-3 + I1) + 1.5*(-1 + J)**2"
NH_prediction_noise1e4 = "0.5*(-3 + I1) + 1.5*(-1 + J)**2"
NH_prediction_noise1e3 = "0.5*(-3 + I1) + 1.5*(-1 + J)**2"

IS_prediction_noise1e0 = "-3 + I1 + 0.5*(-3 + I2) + (-3 + I1)**2 + 1.5*(-1 + J)**2"
IS_prediction_noise1e4 = "-3 + I1 + 0.5*(-3 + I2) + (-3 + I1)**2 + 1.5*(-1 + J)**2"
IS_prediction_noise1e3 = "1.5*(-3 + I2) + (-3 + I1)**2 + 1.5*(-1 + J)**2"

HW_prediction_noise1e0 = " 1.5*(-3 + I2) + (-3 + I1)**2 + 1.5*(-1 + J)**2"
HW_prediction_noise1e4 = "(-3 + I1)*(-3 + I2) + 1.5*(-3 + I2) + 1.5*(-1 + J)**2"
HW_prediction_noise1e3 = "(-3 + I1)*(-3 + I2) + 1.5*(-3 + I2) + 1.5*(-1 + J)**2"

GT_prediction_noise1e0 = "0.75*(-3 + I1) + 1.5*(-1 + J)**2"
GT_prediction_noise1e4 = "0.75*(-3 + I1) + 1.5*(-1 + J)**2"
GT_prediction_noise1e3 = "0.75*(-3 + I1) + 1.5*(-1 + J)**2"
 
OG_prediction_noise1e0 = "0.75*(-3 + I1) + 1.5*(-1 + J)**2"
OG_prediction_noise1e4 = "0.75*(-3 + I1) + 1.5*(-1 + J)**2"
OG_prediction_noise1e3 = "0.75*(-3 + I1) + 1.5*(-1 + J)**2"

def evaluate(expression_string, J_tilde, I1_tilde, I2_tilde):
    J, I1, I2 = se.symbols("J I1 I2")
    sympy_expression = se.sympify(expression_string)
    y_all = np.zeros_like(I1_tilde)
    for i in range(0, I1_tilde.shape[0]):
        y_all[i] = np.array(sympy_expression.subs({J:float(J_tilde[i]), I1:float(I1_tilde[i]), I2:float(I2_tilde[i])}).evalf(16)).astype(np.float128)
    return y_all


# models_names = ['OG']
models_names = ['NH', 'IS', 'HW', 'GT', 'OG']
noise = ['1e0', '1e4', '1e3']
NH_models = [NH_prediction_noise1e0, NH_prediction_noise1e4, NH_prediction_noise1e3]
IS_models = [IS_prediction_noise1e0, IS_prediction_noise1e4, IS_prediction_noise1e3]
HW_models = [HW_prediction_noise1e0, HW_prediction_noise1e4, HW_prediction_noise1e3]
GT_models = [GT_prediction_noise1e0, GT_prediction_noise1e4, GT_prediction_noise1e3]
OG_models = [OG_prediction_noise1e0, OG_prediction_noise1e4, OG_prediction_noise1e3]

model_list = [NH_models, IS_models, HW_models, GT_models, OG_models]
# model_list = [OG_models]

for i, mlist in enumerate(model_list):
    if i == 0:
        expression_string_baseline = NH_baseline
        file  = 'Data/NeoHookean30'
    if i == 1:
        expression_string_baseline = IS_baseline
        file  = 'Data/Isihara30'
    if i == 2:
        expression_string_baseline = HW_baseline
        file  = 'Data/HainesWilson30'
    if i == 3:
        expression_string_baseline = GT_baseline
        file  = 'Data/GentThomas30'
    if i == 4:
        file  = 'Data/Ogden30'


    for j, expression_string_prediction in enumerate(mlist):
        print(file)
        save_file = file + '_noise1e0' +'.npz'
        d30 = np.load(save_file)

        Jarray   = d30['J'][:,0]
        I1array  = d30['I1'][:,0]
        I2array  = d30['I2'][:,0]
        I1array = Jarray**(-2/3)*I1array
        I2array = Jarray**(-4/3)*I2array

        if i != 4:
            y      = evaluate(expression_string_baseline, Jarray, I1array, I2array)
        else:
            y = y_ogden_base
        y_pred = evaluate(expression_string_prediction, Jarray, I1array, I2array)
        error = np.linalg.norm(y-y_pred)/np.linalg.norm(y)
        print("The L2 relative error between the predicted and the %s model for noise level %s is %f"%(models_names[i], noise[j],error))

        # error = np.sqrt(np.mean(y-y_pred)**2)
        # print("The root mean square error between the predicted and the %s model for noise level %s is %f"%(models_names[i], noise[j],error))

Data/NeoHookean30
The L2 relative error between the predicted and the NH model for noise level 1e0 is 0.000000
Data/NeoHookean30
The L2 relative error between the predicted and the NH model for noise level 1e4 is 0.000000
Data/NeoHookean30
The L2 relative error between the predicted and the NH model for noise level 1e3 is 0.000000
Data/Isihara30
The L2 relative error between the predicted and the IS model for noise level 1e0 is 0.011991
Data/Isihara30
The L2 relative error between the predicted and the IS model for noise level 1e4 is 0.011991
Data/Isihara30
The L2 relative error between the predicted and the IS model for noise level 1e3 is 0.011991
Data/HainesWilson30
The L2 relative error between the predicted and the HW model for noise level 1e0 is 0.010539
Data/HainesWilson30
The L2 relative error between the predicted and the HW model for noise level 1e4 is 0.007315
Data/HainesWilson30
The L2 relative error between the predicted and the HW model for noise level 1e3 is 0.007315
Data